In [26]:
import pandas as pd
import pygsheets
import holidays
from datetime import datetime, timedelta

In [27]:
# ORDI FIXE Authentifier avec le fichier de clé JSON
#gc = pygsheets.authorize(service_file=r'D:\DATA\2024-11-2022_Tableau-Heures\feuille-heures-c0ab0678243d.json')

In [28]:
#PC PORTABLE
gc = pygsheets.authorize(service_file=r'D:\Documents\DATA\2024-11-24_CHEMDOC\2024-11-24_Feuille_Heures\Heure_chemdoc\feuille-heures-c0ab0678243d.json')

In [29]:
# Ouvrir la feuille Google Sheet par son titre
#spreadsheet = gc.open('2024-2026_TABLEAU DES HEURES')
spreadsheet = gc.open("RELEVE DES HEURES (REPONSES/NE PAS MODIFIER/FILTRER) ")

In [30]:
# Ouvrir la première feuille (DATA)
worksheet = spreadsheet.sheet1

In [31]:
#Importer toute les valeur et les convertir dans un df pandas
values = worksheet.get_all_values()
df = pd.DataFrame(values[1:], columns=values[0])


In [32]:
df = df.replace('', pd.NA)
df.tail(5)

,Timestamp,"Prénom, NOM",DATE,Lieu,Panier,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,...,Heures travaillées,Affaire,Score,Email Address,,,,,,
11510,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11511,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11512,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11513,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11514,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [33]:
df.head()

,Timestamp,"Prénom, NOM",DATE,Lieu,Panier,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,...,Heures travaillées,Affaire,Score,Email Address,,,,,,
0,30/08/2022,Jonathan EYRAUD,22/08/0022,ATELIER,<NA>,5.00,ANTOFENOL - C2237199,Oui,4.00,LIFE-R-Oasys - LIFE20 ENV/FR/000186,...,<NA>,<NA>,<NA>,jonathan.eyraud@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,25/10/2024,Mehdi EL KANDOUSSI,10/18/0024,ATELIER,<NA>,7,GRAND LAC - REUT - 24222172,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,mehdi.el-kandoussi@chemdocwater.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,03/05/2024,François PIQUET,11/20/2004,BUREAU D'ETUDES,non,7,ISCHIA-KORDIANCE - 22272277,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,piquetfrosty@gmail.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,27/05/2024,François PIQUET,11/20/2004,ATELIER,<NA>,8,DS SMITH - 23201341,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,piquetfrosty@gmail.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,03/06/2024,François PIQUET,11/20/2004,ATELIER,<NA>,8,DS SMITH - 23201341,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,piquetfrosty@gmail.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [34]:
#Supprimer lmes ligne sans valeur dans les colonnes Prénom Nom, Date
df = df.dropna(subset =["Prénom, NOM", "DATE"], how ="all").reset_index(drop =True)
print("Nombre de ligne conservées =",len(df))

Nombre de ligne conservées = 11256


In [35]:
#PREFILTRAGE DES DATE : Convertis la colone timestamp pour reduire la taille du df car la colonne date est trop hétérogène
df["Timestamp"] = pd.to_datetime(df["Timestamp"],format='mixed')

#Créer une colonne anné a partir de timestamp puis nous réduison la taille du DF a partir des date supérieur ou égale 2023
df["Annee_timestamp"] = df["Timestamp"].dt.year
df = df[df["Annee_timestamp"] >= 2023]
print("Nombre de ligne conservées =",len(df))

Nombre de ligne conservées = 7824


In [36]:
#Modifie les années abhérante observée dans date 0024 et 2004
df["DATE"] = df["DATE"].apply(lambda x : x.replace("0024", "2024").replace("2004", "2024"))

In [37]:
#Transforme la colonne date en datetime
df["DATE"] = pd.to_datetime(df["DATE"])

In [38]:
#Créé une colonne année, mois, n) de semaine a partir de la colonne DATE, les ligne 2022 sont peu donc conservées
df["Annee"] = df["DATE"].dt.year
df["Mois"] = df["DATE"].dt.strftime('%m-%B')
df["N_semaine"] = df["DATE"].dt.isocalendar().week
df.head(5)

,Timestamp,"Prénom, NOM",DATE,Lieu,Panier,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,...,,,,,,,Annee_timestamp,Annee,Mois,N_semaine
1,2024-10-25,Mehdi EL KANDOUSSI,2024-10-18,ATELIER,<NA>,7,GRAND LAC - REUT - 24222172,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,10-October,42
2,2024-03-05,François PIQUET,2024-11-20,BUREAU D'ETUDES,non,7,ISCHIA-KORDIANCE - 22272277,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,47
3,2024-05-27,François PIQUET,2024-11-20,ATELIER,<NA>,8,DS SMITH - 23201341,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,47
4,2024-03-06,François PIQUET,2024-11-20,ATELIER,<NA>,8,DS SMITH - 23201341,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,11-November,47
1243,2023-01-27,Lucien LIGOUT,2022-01-13,BUREAU D'ETUDES,non,7.00,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2023,2022,01-January,2


In [39]:
#identification de la list des intérims
list_interim = ['TECHNICIEN MONTEUR1', 'ELECTROTECHNICIEN INDUSTRIEL2', 'MANUTENTIONNAIRE INDUSTRIEL1','ELECTROTECHNICIEN INDUSTRIEL1']

#inverser le prénom et le nom en conservant la liste interim dans le même sens 
def invert_name(name) :
    if name in list_interim :
        return name
    else : 
        invert = ' '.join(name.split()[1:] + [name.split()[0]])
        return invert

df["Prénom, NOM"] = df["Prénom, NOM"].apply(invert_name)
#Renomage de la colonne en NOM_PRENOM
df.rename(columns ={"Prénom, NOM":"NOM_Prénom"}, inplace = True)

In [40]:
#Il y a plusieur colonne heure travaillé, nous renomons la première en HEURE_DECLAREE
df.columns.values[5] = "HEURE_DECLAREE"

In [41]:
#Transfomation de Heure déclaré en float
df["HEURE_DECLAREE"] = df["HEURE_DECLAREE"].astype(float)

In [42]:
#Transformation de la colone panier en binaire 1 = oui pour pouvoir la compter
df["Panier"] = df["Panier"].fillna(0)
df["Panier"] = df["Panier"].apply(lambda x : 1 if x =="oui" else 0)


In [43]:
#Compter les jours travailler si la valeur est différente de 1 = jours travaillé
df["JOUR_TRAVAILLEE"] = df["HEURE_DECLAREE"].apply(lambda x : 0 if x == 0 else 1)

In [44]:
#le DF est aggrégé par Année, Mois , et nuémro de semaine avec une somme par semaine sur heure déclarée et les paniers
df_agg = df.groupby(["Annee","Mois","N_semaine","NOM_Prénom"])[["HEURE_DECLAREE","Panier","JOUR_TRAVAILLEE"]].sum().reset_index()
#Supprésion de l'année 2022
df_agg = df_agg[df_agg["Annee"] > 2022]
#Triage par année et numéro de semaine 
df_agg = df_agg.sort_values(["Annee","NOM_Prénom","N_semaine"])
df_agg

,Annee,Mois,N_semaine,NOM_Prénom,HEURE_DECLAREE,Panier,JOUR_TRAVAILLEE
10,2023,01-January,1,BALDINHO Francisco,35.0,0,5
22,2023,01-January,2,BALDINHO Francisco,35.0,0,5
34,2023,01-January,3,BALDINHO Francisco,35.0,0,5
45,2023,01-January,4,BALDINHO Francisco,35.0,0,5
57,2023,01-January,5,BALDINHO Francisco,14.0,0,2
...,...,...,...,...,...,...,...
1792,2024,10-October,42,VIOLART Sébastien,35.5,0,4
1857,2024,10-October,44,VIOLART Sébastien,35.5,0,4
1895,2024,11-November,45,VIOLART Sébastien,32.0,0,4
1929,2024,11-November,46,VIOLART Sébastien,24.0,0,3


In [45]:
# Obtenir les jours fériés en France pour les années du DataFrame
years = df_agg["Annee"].unique()
french_holidays = holidays.France(years=years)

# Fonction pour calculer les jours ouvrés par semaine
def get_working_days_per_week(year, week):
    start_date = pd.Timestamp(f"{year}-01-01") + pd.Timedelta(weeks=week - 1)
    end_date = start_date + pd.Timedelta(days=6)
    working_days = 0

    for day_offset in range(7):  # Parcourt les 7 jours de la semaine
        current_date = (start_date + pd.Timedelta(days=day_offset)).date()
        # Compte uniquement si c'est un jour ouvré (lundi à vendredi) et non un jour férié
        if current_date.weekday() < 5 and current_date not in french_holidays:
            working_days += 1

    return working_days

# Appliquer la fonction pour chaque ligne
df_agg["Jours_ouvrés"] = df_agg.apply(lambda row: get_working_days_per_week(row["Annee"], row["N_semaine"]), axis=1)

# Calculer le seuil d'heures travaillées en fonction des jours ouvrés
df_agg["Seuil_heures"] = df_agg["Jours_ouvrés"] * 7  # 7 heures par jour ouvré

# Calculer les heures supplémentaires
df_agg["Heures_supplementaires"] = df_agg["HEURE_DECLAREE"] - df_agg["Seuil_heures"]
df_agg["Heures_supplementaires"] = df_agg["Heures_supplementaires"].apply(lambda x: max(x, 0))  # Pas d'heures supp négatives

df_agg


,Annee,Mois,N_semaine,NOM_Prénom,HEURE_DECLAREE,Panier,JOUR_TRAVAILLEE,Jours_ouvrés,Seuil_heures,Heures_supplementaires
10,2023,01-January,1,BALDINHO Francisco,35.0,0,5,5,35,0.0
22,2023,01-January,2,BALDINHO Francisco,35.0,0,5,5,35,0.0
34,2023,01-January,3,BALDINHO Francisco,35.0,0,5,5,35,0.0
45,2023,01-January,4,BALDINHO Francisco,35.0,0,5,5,35,0.0
57,2023,01-January,5,BALDINHO Francisco,14.0,0,2,5,35,0.0
...,...,...,...,...,...,...,...,...,...,...
1792,2024,10-October,42,VIOLART Sébastien,35.5,0,4,5,35,0.5
1857,2024,10-October,44,VIOLART Sébastien,35.5,0,4,4,28,7.5
1895,2024,11-November,45,VIOLART Sébastien,32.0,0,4,5,35,0.0
1929,2024,11-November,46,VIOLART Sébastien,24.0,0,3,4,28,0.0


In [46]:
#Convertis les point en virgule pour google sheet
df_agg = df_agg.applymap( lambda x: f"{x:.2f}".replace('.', ',') if isinstance(x, float) else x)

C:\Users\lucas\AppData\Local\Temp\ipykernel_17352\3436681036.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_agg = df_agg.applymap( lambda x: f"{x:.2f}".replace('.', ',') if isinstance(x, float) else x)


In [47]:
#exportation des données dans google sheet
spreadsheet_export = gc.open('2024-2026_TABLEAU DES HEURES')
#Choisis lapremière feuille
worksheet = spreadsheet_export[0]
#selectionnne la cellule d'arrivé

worksheet.set_dataframe(df_agg, (1, 1))